In [2]:
import numpy as np
import pylab as plt
from numpy import linalg as la
%matplotlib inline

Let $(X,U,P,R)$ be an MDP, i.e.

* $X$ is a finite set of $n$ *states*
* $U$ is a finite set of $m$ *actions*
* $P:X\times U\rightarrow \Sigma(X)$ is transition probability
* $R:X\times U\times X\rightarrow \mathbb{R}$ is cost (for reward, use $-R$)

where $\Sigma(S) = \{p\in[0,1]^S : \sum_{s\in S} p(s) = 1\}$ is the set of probability distributions over the discrete set $S$.

In [10]:
n,m = 5,2
X = range(n)
U = range(m)
# generate transition probabilities
P = np.abs(np.random.randn(n,m,n))
P = P / P.sum(axis=2)[...,np.newaxis]
assert np.all(P.min(axis=2) > 0),'no probability is zero'
assert np.all(P.max(axis=2) < 1),'no probability is one'
assert np.allclose(P.sum(axis=2), 1.),'probabilities sum to 1'
# generate rewards
R = np.abs(np.random.randn(n,m,n))
#R -= R.min(axis=2)[...,np.newaxis]

Starting with an initial state distribution $p\in \Sigma(X)$ and policy $\pi:X\rightarrow \Sigma(U)$, can simulate by iterating:
$$p(x)^+ = \sum_{\xi\in X} p(\xi) \sum_{\mu\in U} \pi(\xi)(\mu) P(\xi,\mu)(x).$$
Equivalently, defining $\Gamma : X \rightarrow \Sigma(X)$ by
$$\forall \xi,x\in X : \Gamma(\xi)(x) = \sum_{\mu\in U} \pi(\xi)(\mu) P(\xi,\mu)(x)$$
and treating $p$ as a row vector, we can iterate $p^+ = p \Gamma$.

In [5]:
# initial state distribution
p0 = np.abs(np.random.randn(n)); p0 = p0 / np.sum(p0);
assert np.allclose(np.sum(p0),1.)
# policy
pi = np.abs(np.random.randn(n,m)); pi = pi / np.sum(pi,axis=1)[:,np.newaxis]
assert np.allclose(np.sum(pi,axis=1),1.)

In [7]:
# iterative simulation
def sim_p(X,U,P,R,p0,pi,T=1):
    n = len(X); m = len(U)
    p_ = [p0]
    for t in range(T):
        p  = p_[-1].copy()
        _p = np.nan*p
        for x in X:
            _p[x] = np.sum([[p[xi] * pi[xi,mu] * P[xi,mu,x] for mu in U] for xi in X])
        assert np.allclose(np.sum(_p),1.),"simulation must preserve probabilities"
        p_.append(_p)
    return p_

# matrix multiplication
G = np.zeros((n,n))
for xi in X:
    for x in X:
        G[xi,x] = np.sum([pi[xi,mu] * P[xi,mu,x] for mu in U])
assert np.allclose(np.sum(G,axis=1),1.)

# sanity check:  matrix multiplication yields same result as iterative simulation
# (note:  p0 is a row vector, so we're doing left multiplication on G)
assert np.allclose(sim_p(X,U,P,R,p0,pi)[-1], np.dot(p0,G))

Since $\Gamma$ is left-stochastic (row sums equal to 1) and ergodic (every state reachable from every other under policy $\pi$), it has one unity eigenvalue, and all other eigenvalues are contractive (magnitude smaller than 1):

In [8]:
print 'spec Gamma =',la.eigvals(G)
assert np.all(np.abs(la.eigvals(G)[1:]) < 1)

spec Gamma = [ 1.00000000+0.j          0.13071453+0.15257207j  0.13071453-0.15257207j
 -0.19463993+0.j         -0.13540749+0.j        ]


This implies that any initial probability distribution will asymptotically converge to the eigenvector corresponding to the unity eigenvalue, which will have no zero entries:

In [11]:
v = la.eig(G.T)[1][:,0]; v /= v.sum()
assert np.allclose(v,np.dot(v,G)),'v is left-eigvec of Gamma with eigval 1'
assert np.allclose(v, np.dot(p0,la.matrix_power(G,100))),'p0 converges to v'
assert np.all(v > 0.),'all states have positive probability'

Let us now consider the problem of minimizing the infinite-horizon discounted cost
$$J = \sum_{t=0}^\infty \gamma^t r_t,$$
where $\gamma\in(0,1)$ is a *discount factor* and $r_t = R(x_t,u_t)$ is the reward at time $t$.

In [12]:
g = np.random.rand()

Any policy $\pi : X\rightarrow\Sigma(U)$ has an associated *value* $V^\pi : X\rightarrow\mathbb{R}$ defined by
$$\forall \xi\in X : V^\pi(\xi) = E[J \mid x = \xi]$$
that satisfies the *Bellman equation*
$$\forall\xi\in X : V^\pi(\xi) = \sum_{\mu\in U}\pi(\xi,\mu)\sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma V^\pi(x)].$$
Noting that $V^\pi$ appears linearly in the above equation, we can solve for $V^\pi$ given $\pi$ using linear algebra:
$$L V^\pi = b,$$
$$b(\xi) = \sum_{x\in X}\sum_{\mu\in U} \pi(\xi)(\mu) P(\xi,\mu)(x) R(\xi,\mu,x),$$

$$L(\xi,x) = \delta(\xi,x) - \sum_{\mu\in U} g \pi(\xi)(\mu) P(\xi,\mu)(x),$$
where $\delta:X\times X\rightarrow\{0,1\}$ is the *Kronecker delta*, i.e. $\delta(\xi,x) = 1 \iff \xi = x$.

In [13]:
def policy_evaluation(X,U,P,R,g,pi):
    b = np.asarray([ np.sum([[pi[xi,mu] * P[xi,mu,x] * R[xi,mu,x] 
                          for x in X] for mu in U]) for xi in X])
    I = np.identity(n)
    L = I - np.asarray([ np.asarray([ np.sum([g * pi[xi,mu] * P[xi,mu,x] 
                                              for mu in U]) for x in X]) for xi in X])
    # V satisfies Bellman equation
    V = np.dot(b,la.inv(L).T)
    assert np.allclose(V, [np.sum([[pi[xi,mu] * P[xi,mu,x] * (R[xi,mu,x] + g*V[x]) 
                                for x in X] for mu in U]) for xi in X])
    return V
print 'value of policy:'
print policy_evaluation(X,U,P,R,g,pi)

value of policy:
[ 4.98078606  4.78922657  4.41806804  4.85122071  4.54070212]


Given the value $V^\pi:X\rightarrow\mathbb{R}$ of any policy $\pi:X\rightarrow\Sigma(U)$, we can perform *policy improvement* via:
$$\forall\xi\in X : \pi'(\xi) = \arg\min_{\mu\in U}\sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma V^\pi(x)].$$
(Note that there is a slight abuse of notation here since $\pi'(\xi)\in\Sigma(U)$ not $\pi'(\xi)\in U$; the equation should be interpreted as specifying a deterministic policy.)

In [14]:
def policy_improvement(X,U,P,R,g,V):
    n = len(X); m = len(U)
    pi = np.zeros((n,m))
    for xi in X:
        u = np.argmin([np.sum([P[xi,mu,x]*(R[xi,mu,x] + g*V[x]) for x in X]) for mu in U])
        #assert len(u) == 1
        pi[xi,u] = 1.
    return pi

In [15]:
print 'improved policy:'
print policy_improvement(X,U,P,R,g,policy_evaluation(X,U,P,R,g,pi))

improved policy:
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]


Iteratively evaluating and improving the policy defines a *policy iteration* algorithm that converges to the optimal deterministic policy:

In [16]:
def policy_iteration(X,U,P,R,g,pi0):
    pi_ = [pi0]
    V_  = []
    while len(V_) < 2 or not(np.allclose(V_[-1],V_[-2])):
        pi = pi_[-1]
        V  = policy_evaluation(X,U,P,R,g,pi)
        _pi = policy_improvement(X,U,P,R,g,V)
        pi_.append(_pi)
        V_.append(V)
    return pi_,V_

pi_PI,V_PI = policy_iteration(X,U,P,R,g,pi)
print '%d iterations' % len(V_PI)
print 'optimal policy:'
print pi_PI[-1]
print 'optimal value:'
print np.asarray(V_PI[-1])

3 iterations
optimal policy:
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]
optimal value:
[ 4.49335097  4.30066824  4.04112354  4.43559621  4.07243505]


Rather than evaluate the value of the policy, we can iterate toward the optimal policy and value simultaneously:
$$\forall\xi\in X : V_{j+1}(\xi) = \sum_{\mu\in U}\pi_j(\xi,\mu)\sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma V_j(x)].$$
$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U}\sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma V_{j+1}(x)].$$

In [17]:
def value_update(X,U,P,R,g,pi,V):
    return np.asarray([np.sum([[pi[xi,mu] * P[xi,mu,x] * (R[xi,mu,x] + g * V[x]) 
                                for x in X] for mu in U]) for xi in X])

In [18]:
def value_iteration(X,U,P,R,g,pi0,maxiter=100):
    n = len(X)
    pi_ = [pi0]
    V_  = [np.zeros(n)]
    while len(V_) < 2 or (not(np.allclose(V_[-1],V_[-2])) and len(V_) < maxiter):
        pi = pi_[-1]
        V  = V_[-1]
        _V = value_update(X,U,P,R,g,pi,V)
        _pi = policy_improvement(X,U,P,R,g,_V)
        pi_.append(_pi)
        V_.append(_V)
    return pi_,V_

pi_VI,V_VI = value_iteration(X,U,P,R,g,pi,200)
print '%d iterations' % len(V_VI)
print 'optimal policy:'
print pi_VI[-1]
print 'optimal value:'
print np.asarray(V_VI[-1])

54 iterations
optimal policy:
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]
optimal value:
[ 4.4931931   4.30051037  4.04096567  4.43543835  4.07227718]


In [19]:
print 'policy discrepancy between VI and PI = %d'%np.max(np.abs(pi_VI[-1] - pi_PI[-1]))
print 'value discrepancy between VI and PI = %0.1e'%np.max(np.abs(V_VI[-1] - V_PI[-1]))

policy discrepancy between VI and PI = 0
value discrepancy between VI and PI = 1.6e-04


Anticipating future methods that learn optimal policies without access to a system model, we formulate policy and value iteration using the *quality* function (also called *action-value* function) $Q:X\times U\rightarrow\mathbb{R}$ defined by:
$$\forall \xi\in X, \mu\in U : Q^\pi(\xi,\mu) = \sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma V^\pi(x)].$$
If $\pi$ is optimal, then $Q^\pi$ is related to $V^\pi$ and $\pi$ via:
$$\forall \xi\in X: V^\pi(\xi) = \min_{\mu\in U} Q^\pi(\xi,\mu),\ \pi(\xi) = \arg\min_{\mu\in U} Q^\pi(\xi,\mu).$$

### Q function policy iteration (11.3-43,44 in LewisVrabieSyrmos2008)
$$\forall\xi\in X,\mu\in U : Q_j(\xi,\mu) = \sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma Q_j(x,\pi_j(x))].$$

$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U} Q_j(\xi,\mu).$$

**This one is more annoying to implement, so I skipped it; I also can't actually make sense of the formula for a nondeterministic policy, due to the \pi_j(x) dependence in the right-hand-side.**

**Here's a proposed alternative formulation that seems to make sense for nondeterministic policies, but the corresponding code doesn't seem to work...**

$$\forall\xi\in X,\mu\in U : Q_j(\xi,\mu) = \sum_{x\in X} P(\xi,\mu)(x)\left[R(\xi,\mu,x) + \gamma \sum_{u\in U}\pi_j(x,u) Q_j(x,u)\right].$$

$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U} Q_j(\xi,\mu).$$

### Q function value iteration (11.3-45,46 in LewisVrabieSyrmos2008)
Just like with *policy iteration* above, iteratively evaluating and improving the policy using the $Q$ function defines a *policy iteration* algorithm:
$$\forall\xi\in X,\mu\in U : Q_{j+1}(\xi,\mu) = \sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma Q_j(x,\pi_j(x)].$$

$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U} Q_{j+1}(\xi,\mu).$$

In [20]:
def value_update_Q(X,U,P,R,g,pi,Q):
    """only correct for deterministic policies"""
    _pi = [np.argmax(pix) for pix in pi]
    return np.asarray([[np.sum([P[xi,mu,x] * (R[xi,mu,x] + g * Q[x,_pi[x]]) 
                                for x in X]) for mu in U] for xi in X])

In [21]:
_pi = pi_VI[-1]

print 'policy value:'
print policy_evaluation(X,U,P,R,g,_pi)

Q = np.zeros((n,m))
for _ in range(10):
    Q = value_update_Q(X,U,P,R,g,_pi,Q)
print 'policy value from Q:'
print Q.min(axis=1)

policy value:
[ 4.49335097  4.30066824  4.04112354  4.43559621  4.07243505]
policy value from Q:
[ 3.86827445  3.67558904  3.41604774  3.81052325  3.44736176]


In [22]:
def policy_improvement_Q(X,U,P,R,g,Q):
    n = len(X); m = len(U)
    pi = np.zeros((n,m))
    for xi in X:
        u = np.argmin(Q[xi])
        pi[xi,u] = 1.
    return pi

In [23]:
def value_iteration_Q(X,U,P,R,g,pi0,maxiter=100):
    n = len(X); m = len(U)
    pi_ = [pi0]
    Q_  = [np.zeros((n,m))]
    while len(Q_) < 2 or (not(np.allclose(Q_[-1],Q_[-2])) and len(Q_) < maxiter):
        pi = pi_[-1]
        Q  = Q_[-1]
        _Q = value_update_Q(X,U,P,R,g,pi,Q)
        _pi = policy_improvement_Q(X,U,P,R,g,_Q)
        pi_.append(_pi)
        Q_.append(_Q)
    return pi_,Q_

pi_VI_Q,Q_VI = value_iteration_Q(X,U,P,R,g,pi,200)
print '%d iterations with Q' % len(V_VI)
print 'optimal policy from Q:'
print pi_VI[-1]
print 'optimal value from Q:'
print Q_VI[-1].min(axis=1)

print 'policy discrepancy between VI and VI with Q = %d'%np.max(np.abs(pi_VI[-1] - pi_VI_Q[-1]))
print 'value discrepancy between VI and VI with Q = %0.1e'%np.max(np.abs(V_VI[-1] - Q_VI[-1].min(axis=1)))

54 iterations with Q
optimal policy from Q:
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]
optimal value from Q:
[ 4.49319084  4.30050811  4.04096341  4.43543608  4.07227492]
policy discrepancy between VI and VI with Q = 0
value discrepancy between VI and VI with Q = 2.3e-06


### Fig 5.1 in Sutton Barto 1998

When the system model (i.e. $P$ and $R$) is not known but trajectory episodes (i.e. $\{(\xi_t^e,\mu_t^e,x_t^e,r_t^e)\mid t\in T,e\in E\}$) are available, can evaluate the policy using "Monte Carlo" methods -- simply compute the average observed discounted future reward for each state.

**Note:** it's important to only include discounted future reward from the first occurence of each state in each simulation trajectory.  (Not sure if violating this slows or prevents convergence . . .)

In [24]:
def sim(X,U,P,R,x0,pi,T=1):
    n = len(X); m = len(U)
    trj = [[None,None,x0,0.]]
    for t in range(T):
        xi = trj[-1][2]
        mu = np.random.choice(U,p=pi[xi])
        x  = np.random.choice(X,p=P[xi,mu])
        r  = R[xi,mu,x]
        trj.append([xi,mu,x,r])
    return trj[1:]

In [25]:
def mc_policy_evaluation(X,trjs,g):
    n = len(X)
    V = np.zeros(n)
    N = np.zeros(n)
    for trj in trjs:
        x_ = []
        T = len(trj)
        r_ = np.asarray(trj)[:,-1]
        g_ = g**np.asarray(range(T))
        for t,(xi,mu,x,r) in enumerate(trj):
            if xi not in x_:
                V[xi] += np.sum(r_[t:]*g_[:T-t])
                N[xi] += 1
                x_.append(xi)
    return V / N

def mc_policy_evaluation_Q(X,trjs,g):
    n = len(X); m = len(U)
    Q = np.zeros((n,m))
    N = np.zeros((n,m))
    for trj in trjs:
        xu_ = []
        T = len(trj)
        r_ = np.asarray(trj)[:,-1]
        g_ = g**np.asarray(range(T))
        for t,(xi,mu,x,r) in enumerate(trj):
            if (xi,mu) not in xu_:
                Q[xi,mu] += np.sum(r_[t:]*g_[:T-t])
                N[xi,mu] += 1
                xu_.append((xi,mu))
    return Q / (N+1)

In [26]:
E = 2000 # number of episodes
T = 20 # horizon of episodes
trjs = [sim(X,U,P,R,np.random.choice(X),_pi,T=T) for e in range(E)]

In [27]:
print 'value of policy:'
print policy_evaluation(X,U,P,R,g,_pi)
print 'mc value of policy:'
print mc_policy_evaluation(X,trjs,g)
print 'mc value of policy with Q:'
print mc_policy_evaluation_Q(X,trjs,g).max(axis=1) # hack b/c policy is deterministic
print 'Q:'
print Q
print 'mc Q:'
print mc_policy_evaluation_Q(X,trjs,g)

value of policy:
[ 4.49335097  4.30066824  4.04112354  4.43559621  4.07243505]
mc value of policy:
[ 4.16196916  4.08145781  3.71032249  4.2224319   3.84047385]
mc value of policy with Q:
[ 4.15976356  4.07940888  3.70840203  4.22031539  3.83853324]
Q:
[[ 3.98881638  3.86827445]
 [ 3.67558904  3.78871182]
 [ 3.41604774  3.43368265]
 [ 3.89389104  3.81052325]
 [ 3.44736176  3.59903999]]
mc Q:
[[ 0.          4.15976356]
 [ 4.07940888  0.        ]
 [ 3.70840203  0.        ]
 [ 0.          4.22031539]
 [ 3.83853324  0.        ]]


For a nondeterministic policy, using the same number of samples as above is clearly insufficient:

In [28]:
E = 2000 # number of episodes
T = 20 # horizon of episodes
trjs = [sim(X,U,P,R,np.random.choice(X),pi,T=T) for e in range(E)]

In [29]:
print 'value of policy:'
print policy_evaluation(X,U,P,R,g,pi)
print 'mc value of policy:'
print mc_policy_evaluation(X,trjs,g)
print 'mc value of policy with Q:'
#print mc_policy_evaluation_Q(X,trjs,g).min(axis=1)
print np.sum(mc_policy_evaluation_Q(X,trjs,g)*pi,axis=1)

value of policy:
[ 4.98078606  4.78922657  4.41806804  4.85122071  4.54070212]
mc value of policy:
[ 4.75160007  4.61513254  3.99250588  4.41523788  3.99179787]
mc value of policy with Q:
[ 4.58234905  4.56783641  3.8017087   4.13941178  3.78376539]


**I'm currently unsure whether I'm using insufficient number or horizon of episodes, have a bug in my code, or this scheme simply doesn't converge for nondeterminstic policies.**